### **1. Inisialisasi dan Konfigurasi Pipeline**

Langkah awal ini adalah fondasi dari keseluruhan pipeline. Di sini, kita melakukan beberapa hal penting:
* **Impor Pustaka**: Mengimpor semua modul yang diperlukan dari TensorFlow (tf), TFX, dan pustaka lainnya. `InteractiveContext` secara khusus digunakan agar kita dapat menjalankan komponen TFX secara interaktif di dalam notebook.
* **Definisi Konstanta**: Menetapkan variabel-variabel global yang akan digunakan di seluruh pipeline. Ini termasuk nama pipeline (`PIPELINE_NAME`), lokasi penyimpanan metadata ML (`METADATA_PATH`), direktori untuk model yang siap di-*deploy* (`SERVING_MODEL_DIR`), dan lokasi data mentah (`DATA_ROOT`). Praktik ini membuat pipeline lebih terstruktur dan mudah dikelola.
* **Konteks Interaktif**: `InteractiveContext` diinisialisasi untuk mengatur lingkungan eksekusi pipeline di dalam notebook ini. Konteks ini akan melacak semua artefak (seperti data, skema, model) dan eksekusi komponen, menyimpannya dalam basis data SQLite di lokasi metadata yang telah ditentukan.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2, pusher_pb2, example_gen_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma

2025-07-09 09:10:34.186663: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 09:10:34.363292: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 09:10:34.943082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-09 09:10:34.943756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-09 09:10:35.075826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import pandas as pd

In [3]:
PIPELINE_NAME = "raffihakim-pipeline"
SCHEMA_PIPELINE_NAME = "student-score-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
DATA_ROOT = 'data'

In [4]:
MODEL_NAME = "score-prediction-model"

In [5]:
serving_model_path = os.path.join(SERVING_MODEL_DIR, MODEL_NAME)

In [6]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [7]:
origin = pd.read_csv("data/student_habits_performance.csv")
origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   student_id                     1000 non-null   object 
 1   age                            1000 non-null   int64  
 2   gender                         1000 non-null   object 
 3   study_hours_per_day            1000 non-null   float64
 4   social_media_hours             1000 non-null   float64
 5   netflix_hours                  1000 non-null   float64
 6   part_time_job                  1000 non-null   object 
 7   attendance_percentage          1000 non-null   float64
 8   sleep_hours                    1000 non-null   float64
 9   diet_quality                   1000 non-null   object 
 10  exercise_frequency             1000 non-null   int64  
 11  parental_education_level       1000 non-null   object 
 12  internet_quality               1000 non-null   ob

### **2. Penyerapan Data (Data Ingestion) dengan CsvExampleGen**

Tahap ini adalah pintu gerbang masuknya data ke dalam pipeline TFX. Komponen yang digunakan adalah **`CsvExampleGen`**.

* **Tujuan**: Membaca data dari sumber eksternal (dalam kasus ini, sebuah file CSV) dan mengubahnya menjadi format `TFRecord`. Format ini sangat efisien untuk digunakan dalam ekosistem TensorFlow.
* **Proses**:
    1.  **`input_base`**: Komponen ini diarahkan ke direktori `DATA_ROOT` ('data/') untuk menemukan file `student_habits_performance.csv`.
    2.  **`output_config`**: Kita mendefinisikan cara data akan dibagi. Di sini, data dibagi menjadi dua set: `train` dan `eval` dengan perbandingan **8:2** (ditentukan oleh `hash_buckets=8` untuk train dan `hash_buckets=2` untuk eval). Pembagian ini penting agar model dilatih pada satu set data dan dievaluasi pada set data lain yang belum pernah dilihatnya.
* **Hasil**: Komponen ini menghasilkan artefak bernama **`Examples`** yang berisi data `train` dan `eval` dalam format TFRecord. Eksekusi melalui `interactive_context.run()` menyimpan artefak ini ke dalam direktori `pipelines/raffihakim-pipeline/CsvExampleGen`.

In [23]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

In [24]:
ingested_data = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(ingested_data)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 35
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Data Validation (StatisticGen)

### **3. Validasi Data**

Validasi data adalah langkah krusial untuk memastikan kualitas dan konsistensi data sebelum masuk ke tahap pelatihan. Proses ini terdiri dari tiga komponen utama:

#### **3.1. StatisticsGen: Menghasilkan Statistik Data**
* **Tujuan**: Menghitung statistik deskriptif untuk setiap fitur dalam dataset. Ini mencakup metrik seperti rata-rata, standar deviasi, nilai minimum/maksimum untuk fitur numerik, serta frekuensi nilai unik untuk fitur kategoris.
* **Proses**: `StatisticsGen` mengambil artefak `Examples` (dari CsvExampleGen) sebagai input dan menganalisis set data `train` dan `eval` secara terpisah.
* **Hasil**: Menghasilkan artefak `ExampleStatistics`. Saat divisualisasikan menggunakan `interactive_context.show()`, kita dapat melihat distribusi, nilai yang hilang, dan properti lainnya dari setiap fitur, yang sangat berguna untuk memahami data secara mendalam.

#### **3.2. SchemaGen: Menginferensi Skema Data**
* **Tujuan**: Membuat skema data secara otomatis berdasarkan statistik yang dihasilkan oleh `StatisticsGen`. Skema ini mendefinisikan tipe data (INT, FLOAT, STRING), valensi (apakah fitur tunggal atau list), dan domain (nilai-nilai yang diharapkan untuk fitur kategoris) dari setiap fitur.
* **Proses**: `SchemaGen` menganalisis artefak `ExampleStatistics` untuk menginferensi properti-properti ini.
* **Hasil**: Menghasilkan artefak `Schema`. Skema ini berfungsi sebagai "kontrak" atau aturan yang harus dipatuhi oleh data. Kita dapat melihat tabel yang merangkum tipe data, keberadaan (apakah wajib ada atau tidak), dan domain untuk setiap fitur.

#### **3.3. ExampleValidator: Memeriksa Anomali Data**
* **Tujuan**: Mengidentifikasi anomali atau kejanggalan dalam data dengan membandingkan statistik data saat ini dengan skema yang telah dibuat.
* **Proses**: `ExampleValidator` mencari masalah seperti:
    * Tipe data yang tidak cocok.
    * Fitur yang seharusnya ada tetapi hilang.
    * Nilai fitur kategoris yang berada di luar domain yang ditentukan.
* **Hasil**: Menghasilkan artefak `ExampleAnomalies`. Dalam kasus ini, output menunjukkan **"No anomalies found,"** yang merupakan hasil ideal. Ini menandakan bahwa data kita bersih, konsisten, dan sesuai dengan skema yang diharapkan.

In [25]:
stats = StatisticsGen(examples = ingested_data.outputs["examples"])
interactive_context.run(stats)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 36
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [26]:
interactive_context.show(stats.outputs['statistics'])

SchemaGen

In [27]:
skema = SchemaGen(statistics=stats.outputs["statistics"])
interactive_context.run(skema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 37
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [28]:
interactive_context.show(skema.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'attendance_percentage',FLOAT,required,,-
'diet_quality',STRING,required,,'diet_quality'
'exam_score',FLOAT,required,,-
'exercise_frequency',INT,required,,-
'extracurricular_participation',STRING,required,,'extracurricular_participation'
'gender',STRING,required,,'gender'
'internet_quality',STRING,required,,'internet_quality'
'mental_health_rating',INT,required,,-


,Values
Domain,
'diet_quality',"'Fair', 'Good', 'Poor'"
'extracurricular_participation',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'internet_quality',"'Average', 'Good', 'Poor'"
'parental_education_level',"'Bachelor', 'High School', 'Master', 'None'"
'part_time_job',"'No', 'Yes'"


ExampleValidator

In [29]:
validing = ExampleValidator(
    statistics=stats.outputs["statistics"],
    schema=skema.outputs["schema"]
)
interactive_context.run(validing)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 38
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [30]:
interactive_context.show(validing.outputs["anomalies"])

### **4. Rekayasa Fitur (Feature Engineering) dengan Transform**

Tahap ini berfokus pada pra-pemrosesan dan transformasi data agar siap digunakan oleh model.
* **Tujuan**: Menerapkan transformasi fitur secara konsisten pada data `train` dan `eval`. Komponen **`Transform`** memastikan bahwa transformasi yang sama yang diterapkan saat pelatihan juga akan diterapkan saat inferensi (prediksi), sehingga menghindari *training-serving skew*.
* **Proses**:
    1.  `Transform` memanggil fungsi `preprocessing_fn` yang didefinisikan dalam file `student_transform.py`.
    2.  Dalam `preprocessing_fn` ini, semua fitur numerik dan label (`exam_score`) dikonversi (di-*cast*) menjadi tipe data `tf.float32`. Meskipun sederhana, ini adalah contoh bagaimana transformasi yang lebih kompleks (seperti normalisasi, pembagian bucket, atau pembuatan *feature cross*) dapat diimplementasikan.
* **Hasil**: Komponen ini menghasilkan dua artefak utama:
    * **`transformed_examples`**: Data `train` dan `eval` yang telah melalui proses transformasi.
    * **`transform_graph`**: Sebuah grafik TensorFlow yang menyimpan logika transformasi. Grafik inilah yang akan digunakan kembali oleh `Trainer`, `Evaluator`, dan saat model di-*deploy* untuk memastikan konsistensi.

In [31]:
TRANSFORM_MODULE_FILE = "student_transform.py"
transform = Transform(
    examples=ingested_data.outputs["examples"],
    schema=skema.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_trainer.py -> build/lib
copying student_transform.py -> build/lib
copying student_tuner.py -> build/lib
installing to /tmp/tmpmkq_dked
running install
running install_lib
copying build/lib/student_transform.py -> /tmp/tmpmkq_dked
copying build/lib/student_trainer.py -> /tmp/tmpmkq_dked
copying build/lib/student_tuner.py -> /tmp/tmpmkq_dked
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpmkq_dked

/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /tmp/tmpmkq_dked/tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4.dist-info/WHEEL
creating '/tmp/tmp7hslz645/tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4-py3-none-any.whl' and adding '/tmp/tmpmkq_dked' to it
adding 'student_trainer.py'
adding 'student_transform.py'
adding 'student_tuner.py'
adding 'tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+044d5b03d7a71df6e04f18dbf0e109fcc568093f0e0137d0b27dcfa2d4ddf9a4.dist-info/RECORD'
removing /tmp/tmpmkq_dked
Processing ./pipelines/raffihakim-pipeline/_wheels/tfx_user_code_Transform-0.0+044d5b03d7a71df6e0

INFO:tensorflow:Assets written to: pipelines/raffihakim-pipeline/Transform/transform_graph/39/.temp_path/tftransform_tmp/774c1fb8ff624a34a4c6ec207fb46551/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 39
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### **5. Pencarian Hyperparameter dengan Tuner**

Sebelum melatih model final, kita perlu menemukan kombinasi hyperparameter terbaik.
* **Tujuan**: Mengotomatiskan proses pencarian hyperparameter yang optimal untuk model kita. Komponen **`Tuner`** secara sistematis mencoba berbagai kombinasi untuk menemukan yang memberikan performa terbaik.
* **Proses**:
    1.  `Tuner` menggunakan fungsi yang didefinisikan dalam `student_tuner.py`.
    2.  **`model_builder`**: Fungsi ini mendefinisikan ruang pencarian (search space) untuk hyperparameter, seperti jumlah lapisan *dense layer* (1 hingga 3), jumlah unit di setiap lapisan (antara 32 dan 256), *dropout rate* (antara 0.1 dan 0.5), dan *learning rate* (0.01, 0.001, atau 0.0001).
    3.  **Strategi**: Menggunakan **`RandomSearch`**, komponen ini secara acak mencoba 10 kombinasi hyperparameter.
    4.  **Objektif**: Tujuannya adalah meminimalkan `val_root_mean_squared_error`.
* **Hasil**:
    * `Tuner` mengeluarkan artefak **`best_hyperparameters`**, yang berisi kombinasi hyperparameter dengan skor evaluasi terbaik.
    * Hasilnya menunjukkan bahwa model terbaik memiliki 2 lapisan, 96 dan 224 unit, dropout 0.5, dan learning rate 0.001, dengan **RMSE sekitar 4.65**.

In [32]:
TUNER_MODULE_FILE = "student_tuner.py"
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=skema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=100)
)
interactive_context.run(tuner)

Trial 10 Complete [00h 00m 53s]
val_root_mean_squared_error: 4.953655242919922

Best val_root_mean_squared_error So Far: 4.953655242919922
Total elapsed time: 00h 07m 38s
Results summary
Results in pipelines/raffihakim-pipeline/.temp/40/student_performance_tuning
Showing 10 best trials
Objective(name="val_root_mean_squared_error", direction="min")

Trial 09 summary
Hyperparameters:
num_layers: 1
units_0: 192
dropout: 0.4
learning_rate: 0.001
units_1: 128
units_2: 224
Score: 4.953655242919922

Trial 04 summary
Hyperparameters:
num_layers: 3
units_0: 160
dropout: 0.5
learning_rate: 0.001
units_1: 128
units_2: 96
Score: 5.032444000244141

Trial 07 summary
Hyperparameters:
num_layers: 2
units_0: 256
dropout: 0.2
learning_rate: 0.01
units_1: 32
units_2: 128
Score: 5.041709899902344

Trial 01 summary
Hyperparameters:
num_layers: 3
units_0: 64
dropout: 0.2
learning_rate: 0.01
units_1: 32
units_2: 32
Score: 5.0759663581848145

Trial 06 summary
Hyperparameters:
num_layers: 2
units_0: 96
dropout

ExecutionResult(
    component_id: Tuner
    execution_id: 40
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### **6. Pelatihan Model dengan Trainer**

Ini adalah inti dari proses machine learning, di mana model sebenarnya dilatih.
* **Tujuan**: Melatih model Neural Network menggunakan data yang telah ditransformasi dan hyperparameter terbaik yang ditemukan oleh `Tuner`.
* **Proses**:
    1.  Komponen **`Trainer`** mengambil `transformed_examples`, `transform_graph`, `schema`, dan `best_hyperparameters` sebagai input.
    2.  Logika pelatihan didefinisikan dalam file `student_trainer.py`. Fungsi `_build_keras_model` membangun arsitektur model berdasarkan parameter yang diterima, dan `run_fn` menjalankan proses `model.fit`.
    3.  Model dilatih selama 15 *epoch* pada data `train` dan divalidasi pada data `eval` di setiap akhir *epoch*.
* **Hasil**: Komponen ini menghasilkan artefak `Model` yang berisi model terlatih yang siap untuk dievaluasi lebih lanjut atau di-*deploy*.Trainer

In [33]:
TRAINER_MODULE_FILE = "student_trainer.py"
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=skema.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=100),
)
interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_trainer.py -> build/lib
copying student_transform.py -> build/lib
copying student_tuner.py -> build/lib
installing to /tmp/tmp4adxrzxd
running install
running install_lib
copying build/lib/student_transform.py -> /tmp/tmp4adxrzxd
copying build/lib/student_trainer.py -> /tmp/tmp4adxrzxd
copying build/lib/student_tuner.py -> /tmp/tmp4adxrzxd
running install_egg_info


/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp4adxrzxd/tfx_user_code_Trainer-0.0+3406cfa487f5e195e132dcecc7c52741d0ec0e72e6f341abbb062d59551e1a89.egg-info
running install_scripts
creating /tmp/tmp4adxrzxd/tfx_user_code_Trainer-0.0+3406cfa487f5e195e132dcecc7c52741d0ec0e72e6f341abbb062d59551e1a89.dist-info/WHEEL
creating '/tmp/tmpy6ssonqx/tfx_user_code_Trainer-0.0+3406cfa487f5e195e132dcecc7c52741d0ec0e72e6f341abbb062d59551e1a89-py3-none-any.whl' and adding '/tmp/tmp4adxrzxd' to it
adding 'student_trainer.py'
adding 'student_transform.

Processing ./pipelines/raffihakim-pipeline/_wheels/tfx_user_code_Trainer-0.0+3406cfa487f5e195e132dcecc7c52741d0ec0e72e6f341abbb062d59551e1a89-py3-none-any.whl
Hyperparameters Terbaik yang Digunakan: {'num_layers': 1, 'units_0': 192, 'dropout': 0.4, 'learning_rate': 0.001, 'units_1': 128, 'units_2': 224}
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 study_hours_per_day_xf (In  [(None, 1)]                  0         []                            
 putLayer)                                                                                        
                                                                                                  
 social_media_hours_xf (Inp  [(None, 1)]                  0         []                            
 utLayer)                                                                           

INFO:tensorflow:Assets written to: pipelines/raffihakim-pipeline/Trainer/model/41/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 41
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Resolver

In [34]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 42
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### **7. Evaluasi dan Validasi Model dengan Evaluator**

Setelah model dilatih, kita perlu mengevaluasi performanya secara mendalam.
* **Tujuan**: Menganalisis kinerja model terlatih pada set data evaluasi. **`Evaluator`** tidak hanya menghitung metrik keseluruhan, tetapi juga memungkinkan analisis pada irisan (slice) data yang berbeda.
* **Proses**:
    1.  **`Resolver`**: Pertama, `Resolver` dengan strategi `LatestBlessedModelStrategy` digunakan untuk mencari model "blessed" (model yang sudah lolos validasi sebelumnya) sebagai *baseline*. Pada eksekusi pertama, tidak ada *baseline* yang ditemukan.
    2.  **`Evaluator`**: Komponen ini mengambil model baru (dari `Trainer`) dan data evaluasi.
    3.  **`EvalConfig`**: Kita mendefinisikan konfigurasi evaluasi untuk menghitung metrik `ExampleCount`, `MeanSquaredError`, dan `RootMeanSquaredError` pada keseluruhan data.
* **Hasil**:
    * `Evaluator` menghasilkan artefak **`evaluation`** yang berisi hasil analisis metrik.
    * Komponen ini juga menghasilkan artefak **`blessing`**. `blessing` ini menentukan apakah model baru "diberkati" (dianggap cukup baik untuk produksi) atau tidak. Karena tidak ada *baseline*, model pertama secara default akan diberkati.
    * Visualisasi `render_slicing_metrics` menunjukkan bahwa RMSE model pada data evaluasi adalah sekitar **18.00**.Evaluator

In [63]:
import tensorflow_model_analysis as tfma

simple_eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='exam_score_xf')
    ],
    slicing_specs=[
        tfma.SlicingSpec()
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            thresholds={
                # Metrik yang dievaluasi: root_mean_squared_error
                'root_mean_squared_error': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        # Model dianggap 'BLESSED' jika RMSE di bawah 8.0
                        upper_bound={'value': 8.0} 
                    )
                )
            },
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError'),
                tfma.MetricConfig(class_name='RootMeanSquaredError')
            ]
        )
    ]
)

In [65]:
evaluator_simple = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=simple_eval_config
)
interactive_context.run(evaluator_simple)

ExecutionResult(
    component_id: Evaluator
    execution_id: 46
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

TFMA Visualization

In [66]:
eval_result = evaluator_simple.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

### **8. Penyiapan Model untuk Deployment (Pusher)**

Langkah terakhir dalam pipeline ini adalah memindahkan model yang telah divalidasi ke lokasi yang dapat diakses untuk inferensi atau penyajian (*serving*).
* **Tujuan**: "Mendorong" (*push*) model yang telah terbukti bagus (berdasarkan artefak `blessing`) ke direktori *deployment*.
* **Proses**:
    1.  Komponen **`Pusher`** memeriksa artefak `blessing` dari `Evaluator`.
    2.  Jika model diberkati (`blessed`), `Pusher` akan menyalin artefak model dari `Trainer` ke lokasi yang ditentukan dalam `push_destination`.
    3.  Dalam kasus ini, `push_destination` adalah direktori `serving_model/raffihakim-pipeline/score-prediction-model`.
* **Hasil**: Artefak model sekarang tersedia di direktori `serving_model`. Model ini memiliki struktur yang siap untuk di-*load* oleh platform penyajian seperti **TensorFlow Serving**, sehingga dapat digunakan untuk membuat prediksi pada data baru.

In [67]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_path
        )
    ))

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 47
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))